In [1]:
import sys
sys.path.append('..')
from tqdm import tqdm

import torch
import torch.nn as nn
import matplotlib.pyplot as plt 

from models.transformer import TimeSeriesTransformerAutoencoder

In [2]:
from dataset.cycling_dataset import TorchCyclingDataset
from torch.utils.data import DataLoader

train_dataset = TorchCyclingDataset(
    file_path_acc='/Users/Aleyna/downloads/preprocessed/handlebar_acc_train.h5',
    file_path_gyro='/Users/Aleyna/downloads/preprocessed/handlebar_gyro_train.h5',
)

test_dataset = TorchCyclingDataset(
    file_path_acc='/Users/Aleyna/downloads/preprocessed/handlebar_acc_test.h5',
    file_path_gyro='/Users/Aleyna/downloads/preprocessed/handlebar_gyro_test.h5',
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [4]:
next(iter(train_loader)).__class__

list

In [51]:
model = TimeSeriesTransformerAutoencoder(input_dim=6, model_dim=16, num_heads=4, num_layers=4, seq_length=100)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

train_losses = []
val_losses = []

for epoch in range(1):
    model.train()
    for x, _ in tqdm(train_loader):
        x_rec = model(x)
        loss = torch.sqrt(criterion(x_rec, x)) # RMSE (reconstruction error)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    model.eval()
    with torch.no_grad():
        for i, (x, _) in enumerate(test_loader):
            x_rec = model(x)
            loss = torch.sqrt(criterion(x_rec, x))
            val_losses.append(loss.item())

    torch.save(model.state_dict(), 'transformer_autoencoder.pth')

100%|██████████| 154/154 [01:43<00:00,  1.49it/s]


In [53]:
model.encode(torch.randn(1, 100, 6)).shape

torch.Size([1, 100, 16])